In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge,Lasso,ElasticNet, RidgeCV,LassoCV,ElasticNetCV
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor,RandomForestRegressor
import sklearn.metrics as mt
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import PolynomialFeatures,StandardScaler

In [15]:
data = pd.read_csv("advertising.csv")
df = data.copy()

y = df["Sales"]
X = df.drop("Sales", axis=1)

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1,random_state=42)

#scy = StandardScaler()
#scx = StandardScaler()

#X_train = scx.fit_transform(X_train)
#X_test = scy.fit_transform(X_test)

class Regression():
    def __init__(self,X_train,X_test,y_train,y_test,X,y):
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test
        self.X = X
        self.y = y
    def basla(self):
        self.all_regression()
        pd.options.display.float_format = '{:,.4f}'.format
        self.sonuclar = pd.DataFrame(self.sonuclar,
                                     columns=["Model","R2","RMSE","MSE","Doğrulama","Polinom Derecesi"]
                                    ).sort_values(by='R2', ascending=False)
        return self.sonuclar
    
    def all_regression(self):
        #RANDOM FOREST REGRESSOR
        self.randomforest()
        self.rf_model = RandomForestRegressor(random_state=0,
                                              max_depth=self.max_depth_value,
                                              max_features=self.max_features_value,
                                              n_estimators= self.n_estimators_rf
                                             )
        self.rf_model.fit(self.X_train,self.y_train)
        self.rf_tahmin = self.rf_model.predict(self.X_test)
        
        self.sonuclar = [
            ("RandomForestRegressor",
                          self.hata_test(self.rf_tahmin)[2],
                          self.hata_test(self.rf_tahmin)[1],
                          self.hata_test(self.rf_tahmin)[0],
                          self.caprazdog(self.rf_model),
                         )
           ]
    def hata_test(self,tahmin):
        self.mse = mt.mean_squared_error(self.y_test,tahmin,squared=True)
        self.rmse = mt.mean_squared_error(self.y_test,tahmin,squared=False)
        self.r2 = mt.r2_score(self.y_test,tahmin)
        return [self.mse,self.rmse,self.r2]
    
    def caprazdog(self,model):
        self.dogruluk = cross_val_score(model,self.X,self.y,cv=10)
        return self.dogruluk.mean()
    
    def hyperparameter(self):
        def ridge():
            self.lambdalar = 10**np.linspace(10,-2,100)*0.5
            self.ridge_lamb = RidgeCV(alphas=self.lambdalar, scoring="r2").fit(self.X_train,self.y_train).alpha_
        def lasso():
            self.lasso_lamb = LassoCV(cv=10,max_iter=10000).fit(self.X_train,self.y_train).alpha_
        def elasticnet():
            self.elastic_lamb = ElasticNetCV(cv=10,max_iter=10000).fit(self.X_train,self.y_train).alpha_
        return ridge(),lasso(),elasticnet()
    def randomforest(self):
        self.rf_model = RandomForestRegressor(random_state=0)
        self.rf_model.fit(self.X_train,self.y_train)
        self.rf_tahmin = self.rf_model.predict(self.X_test)
        
        self.rfparametreler = {
            "max_depth":range(1,3),
            "max_features":range(1,3),
            "n_estimators":range(2,3),
        }
        self.rfgrid = GridSearchCV(estimator=self.rf_model,param_grid=self.rfparametreler,cv=10,n_jobs=-1)
        self.rfgrid.fit(self.X_train,self.y_train)
        self.max_depth_value = self.rfgrid.best_params_['max_depth']
        self.max_features_value = self.rfgrid.best_params_['max_features']
        self.n_estimators_rf = self.rfgrid.best_params_['n_estimators']
        return self.max_depth_value,self.max_features_value,self.n_estimators_rf
    

In [16]:
start = Regression(X_train,X_test,y_train,y_test,X,y)
start.basla()

ValueError: 6 columns passed, passed data had 5 columns